In [1]:
!pip install yt_dlp
!pip install openai

     ---------------------------------------- 0.0/172.2 kB ? eta -:--:--
     -- ------------------------------------- 10.2/172.2 kB ? eta -:--:--
     -- ------------------------------------- 10.2/172.2 kB ? eta -:--:--
     ---- -------------------------------- 20.5/172.2 kB 110.1 kB/s eta 0:00:02
     ---- -------------------------------- 20.5/172.2 kB 110.1 kB/s eta 0:00:02
     -------- ---------------------------- 41.0/172.2 kB 164.3 kB/s eta 0:00:01
     ------------- ----------------------- 61.4/172.2 kB 218.8 kB/s eta 0:00:01
     ---------------------------------- - 163.8/172.2 kB 517.2 kB/s eta 0:00:01
     ------------------------------------ 172.2/172.2 kB 519.2 kB/s eta 0:00:00
   ---------------------------------------- 0.0/3.2 MB ? eta -:--:--
   --------- ------------------------------ 0.7/3.2 MB 15.5 MB/s eta 0:00:01
   ------------------- -------------------- 1.6/3.2 MB 17.0 MB/s eta 0:00:01
   ------------------------------------- -- 3.0/3.2 MB 21.7 MB/s eta 0:00:01

In [ ]:
import re
api_key = 

URLS = [
    "https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9",
    "https://youtu.be/ekr2nIex040?si=rRjc8j1PLoBzttmc",
    "https://youtu.be/qrO4YZeyl0I",
    "https://youtu.be/VG3JsmOmDqw",
    "https://www.youtube.com/watch?v=lV6ppJbzQQM",
    "https://youtu.be/L4IR6SkE_Aw?si=xQaKoIx4sxUTENBf",
    "https://youtu.be/Lg_Pn45gyMs?si=HrnvZgkJD2FjyosH",
    "https://www.youtube.com/watch?v=t9HR_bVmYN0",
    "https://youtu.be/nJ6A6GC_ki4",
    "https://youtu.be/aW7bzd8uwyQ?si=IZ5ShKeBcJKFu-r6",
    "https://youtu.be/UOxkGD8qRB4?si=oYfxd33amgpmiDBN",
    "https://www.youtube.com/watch?v=fsxVo6WqI8w",
    "https://youtu.be/mo7MpQZRd5Q?si=CikTFRVNPdvvLaqH",
    "https://youtu.be/lDK9QqIzhwk",
    "https://www.youtube.com/watch?v=eBSQug_xtwc",
    "https://youtu.be/GJtrE-SLtZA?si=w7xuf3hBToB5V7Au",
    "https://youtu.be/VZzSBv6tXMw?si=nylwJqJ8ewjfspgC",
    "https://youtu.be/CB2_qBzSft8?si=YRAwmXCg0n_YjOfq",
    "https://youtu.be/QvypZSdjO8M?si=A0dmPbAAYK5rwbLM",
    "https://www.youtube.com/watch?v=GJyqi3nNadg",
    "https://youtu.be/QCZZwZQ4qNs?si=WKU7kNcj8ICKMvwD",
    "https://youtu.be/UMYAdGEXOn0?si=sI8VRdbbWDXu4lEt",
    "https://youtu.be/OPf0YbXqDm0?si=NlaIprnObgFH2aIy",
    "https://youtu.be/fmI_Ndrxy14?si=mJyeXSsO-8UgzdVz",
    "https://youtu.be/DMKX9ETeJOw?si=ZXXlxokoAKGz6HHe",
    "https://youtu.be/fmI_Ndrxy14?si=c4L1dXH-LDCgGMvu",
    "https://youtu.be/H7k3ma7YNmo?si=XLt4kzkbnMxrzlaP",
    "https://youtu.be/UsnRQJxanVM?si=RI4EqEjIo_0Q_7v7",
    "https://youtu.be/s-Dq5FJEH10?si=fIhFAgx1JxL7UZu5",
    "https://youtu.be/ZfR_FP_7w00?si=N9WjmPEcREP53YOJ",
]



In [ ]:
import yt_dlp
import csv
import os
import openai
import time

def format_seconds_to_mmss(seconds):
    minutes, secs = divmod(int(seconds), 60)
    return f"{minutes:02d}:{secs:02d}"

OUTPUT_FOLDER = "songs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CSV_PATH = "songs_info.csv"

def download_song_and_metadata(url, output_folder):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_folder + '/%(title)s.%(ext)s',
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en', 'es','auto'],  # Try to get subs in EN, ES
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'noplaylist': True,
        'skip_download': False,
        'forcefilename': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

        title = info.get('title')
        duration = info.get('duration')
        # Subtitle
        subtitle_text = None
        if info.get('automatic_captions') or info.get('subtitles'):
            subs = info.get('subtitles') or info.get('automatic_captions')
            lang_code = next(iter(subs)) if subs else None
            if lang_code:
                sub_info = subs[lang_code][0]  # take the first available
                sub_url = sub_info['url']
                # Download subtitle file as .vtt
                import requests
                r = requests.get(sub_url)
                sub_filename = os.path.join(output_folder, f"{title}.vtt")
                with open(sub_filename, "wb") as f:
                    f.write(r.content)
                # Parse .vtt txt
                with open(sub_filename, "r", encoding="utf-8") as f:
                    vtt = f.read()
                # Remove timestamps and get text only
                lines = []
                for l in vtt.splitlines():
                    if "-->" in l or l.strip() == "" or l[0].isdigit():
                        continue
                    lines.append(l)
                subtitle_text = " ".join(lines)
        return title, duration, subtitle_text

# Initialize the OpenAI client with your API key and the latest API version
client = openai.OpenAI(api_key=api_key)

def gpt_infer(subtitle_text):
    prompt = (
        "Dada la letra de la canción que te estoy pasando, identifica lo siguiente:"
        "1. Tematica:  (Latino/Anime/JPOP/KPOP/Videojuegos/Clasicos/Conocidas/Otras), "
        "2. Idioma hablado"
        "3. Genero musical (e.g., pop, reggaeton, españolas, rap, classical, etc.). "
        "Return your answer as a CSV row in the format: Genero,Idioma,Tematica.\n\n"
        "Transcript:\n"
        f"{subtitle_text}\n"
        "CSV:"
    )
    try:
        response = client.chat.completions.create(
            model="GPT-4.1 nano",
            messages=[{"role": "user", "content": prompt}],
        )
        txt = response.choices[0].message.content.strip()
        fields = txt.split(",")
        if len(fields) == 3:
            genre, language, thematic = [f.strip() for f in fields]
            return genre, language, thematic
        else:
            return "Unknown", "Unknown", "Unknown"
    except Exception as e:
        print(f"OpenAI API error: {e}")
        return "Unknown", "Unknown", "Unknown"


def main():
    with open(CSV_PATH, "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Name","Length","Genre","Language","Thematic"])
        for url in URLS:
            print(f"\nProcessing: {url}")
            title, duration, subtitle = download_song_and_metadata(url, OUTPUT_FOLDER)
            print(f"Downloaded: {title}")
            if subtitle and len(subtitle) > 100:
                genre, language, thematic = gpt_infer(subtitle[:2000])
                print(f"AI Inference: Genre={genre}, Language={language}, Thematic={thematic}")
            else:
                genre, language, thematic = "Unknown", "Unknown", "Unknown"
                print("No suitable subtitles/lyrics for inference.")
            writer.writerow([title, format_seconds_to_mmss(duration), genre, language, thematic])
            time.sleep(2)  # be gentle with the API

    print(f"\nDone! Output in: {CSV_PATH}")

if __name__ == "__main__":
    main()



Processing: https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9
songs/Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas.webm
Downloaded: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas
No suitable subtitles/lyrics for inference.

Processing: https://youtu.be/ekr2nIex040?si=rRjc8j1PLoBzttmc
songs/ROSÉ & Bruno Mars - APT. (Official Music Video).webm
Downloaded: ROSÉ & Bruno Mars - APT. (Official Music Video)
OpenAI API error: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}
AI Inference: Genre=Unknown, Language=Unknown, Thematic=Unknown

Processing: https://youtu.be/qrO4YZeyl0I
songs/Lady Gaga - Bad Romance (Official Music Video).webm
Downloaded: Lady Gaga - Bad Romance (Official Music Video)
OpenAI API error: Error code: 400 - {'error': {'message': 'invalid model ID', 'type': 'invalid_request_error', 'param': None, 'code': None}}
AI Inference: Genre=Unknown, Language=Unknown, Thematic=Unk

FileNotFoundError: [Errno 2] No such file or directory: 'songs/K/DA - POP/STARS (ft. Madison Beer, (G)I-DLE, Jaira Burns) | Music Video - League of Legends.vtt'

In [ ]:
import yt_dlp
import csv
import os
import openai
import time
import requests

def format_seconds_to_mmss(seconds):
    minutes, secs = divmod(int(seconds), 60)
    return f"{minutes:02d}:{secs:02d}"


OUTPUT_FOLDER = "songs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CSV_PATH = "songs_info.csv"

def search_youtube_for_better_version(title):
    query = title + " official audio high quality sound"
    search_url = f"ytsearch10:{query}"  # Search top 10 results
    ydl_opts = {'quiet': True, 'skip_download': True}

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        results = ydl.extract_info(search_url, download=False)
        videos = results['entries']
        if not videos:
            return None

        # First, try to find a video with subtitles and full metadata
        for video in videos:
            if video.get('subtitles') or video.get('automatic_captions'):
                if all(key in video for key in ['title', 'duration', 'uploader']):
                    print(f"Selected: {video['title']} from {video['uploader']}")
                    return f"https://www.youtube.com/watch?v={video['id']}"

        # If no video with subs+metadata, fall back to first
        best_video = videos[0]
        print(f"Fallback Selected: {best_video['title']}")
        return f"https://www.youtube.com/watch?v={best_video['id']}"

def download_song_and_metadata(url, output_folder):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': output_folder + '/%(title)s.%(ext)s',
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en', 'es', 'auto'],
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'noplaylist': True,
        'skip_download': False,
        'forcefilename': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

        title = info.get('title')
        duration = info.get('duration')
        subtitle_text = None

        if info.get('automatic_captions') or info.get('subtitles'):
            subs = info.get('subtitles') or info.get('automatic_captions')
            lang_code = next(iter(subs)) if subs else None
            if lang_code:
                sub_info = subs[lang_code][0]
                sub_url = sub_info['url']
                r = requests.get(sub_url)
                sub_filename = os.path.join(output_folder, f"{title}.vtt")
                with open(sub_filename, "wb") as f:
                    f.write(r.content)
                with open(sub_filename, "r", encoding="utf-8") as f:
                    vtt = f.read()
                lines = []
                for l in vtt.splitlines():
                    if "-->" in l or l.strip() == "" or l[0].isdigit():
                        continue
                    lines.append(l)
                subtitle_text = " ".join(lines)
        return title, duration, subtitle_text

client = openai.OpenAI(api_key=api_key)

def gpt_infer(subtitle_text):
    prompt = (
        "Dada la letra de la canción que te estoy pasando, identifica lo siguiente:"
        "1. Tematica:  (Latino/Anime/JPOP/KPOP/Videojuegos/Clasicos/Conocidas/Otras), "
        "2. Idioma hablado"
        "3. Genero musical (e.g., pop, reggaeton, españolas, rap, classical, etc.). "
        "Return your answer as a CSV row in the format: Genero,Idioma,Tematica.\n\n"
        "Transcript:\n"
        f"{subtitle_text}\n"
        "CSV:"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4.1",
            messages=[{"role": "user", "content": prompt}],
        )
        txt = response.choices[0].message.content.strip()
        fields = txt.split(",")
        if len(fields) == 3:
            genre, language, thematic = [f.strip() for f in fields]
            return genre, language, thematic
        else:
            return "Unknown", "Unknown", "Unknown"
    except Exception as e:
        print(f"OpenAI API error: {e}")
        return "Unknown", "Unknown", "Unknown"

# List of original URLs
URLS = [
    "https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9",
    "https://youtu.be/ekr2nIex040?si=rRjc8j1PLoBzttmc",
    "https://youtu.be/qrO4YZeyl0I",
    "https://youtu.be/VG3JsmOmDqw",
    "https://www.youtube.com/watch?v=lV6ppJbzQQM",
    "https://youtu.be/L4IR6SkE_Aw?si=xQaKoIx4sxUTENBf",
    "https://youtu.be/Lg_Pn45gyMs?si=HrnvZgkJD2FjyosH",
    "https://www.youtube.com/watch?v=t9HR_bVmYN0",
    "https://youtu.be/nJ6A6GC_ki4",
    "https://youtu.be/aW7bzd8uwyQ?si=IZ5ShKeBcJKFu-r6",
    "https://youtu.be/UOxkGD8qRB4?si=oYfxd33amgpmiDBN",
    "https://www.youtube.com/watch?v=fsxVo6WqI8w",
    "https://youtu.be/mo7MpQZRd5Q?si=CikTFRVNPdvvLaqH",
    "https://youtu.be/lDK9QqIzhwk",
    "https://www.youtube.com/watch?v=eBSQug_xtwc",
    "https://youtu.be/GJtrE-SLtZA?si=w7xuf3hBToB5V7Au",
    "https://youtu.be/VZzSBv6tXMw?si=nylwJqJ8ewjfspgC",
    "https://youtu.be/CB2_qBzSft8?si=YRAwmXCg0n_YjOfq",
    "https://youtu.be/QvypZSdjO8M?si=A0dmPbAAYK5rwbLM",
    "https://www.youtube.com/watch?v=GJyqi3nNadg",
    "https://youtu.be/QCZZwZQ4qNs?si=WKU7kNcj8ICKMvwD",
    "https://youtu.be/UMYAdGEXOn0?si=sI8VRdbbWDXu4lEt",
    "https://youtu.be/OPf0YbXqDm0?si=NlaIprnObgFH2aIy",
    "https://youtu.be/fmI_Ndrxy14?si=mJyeXSsO-8UgzdVz",
    "https://youtu.be/DMKX9ETeJOw?si=ZXXlxokoAKGz6HHe",
    "https://youtu.be/fmI_Ndrxy14?si=c4L1dXH-LDCgGMvu",
    "https://youtu.be/H7k3ma7YNmo?si=XLt4kzkbnMxrzlaP",
    "https://youtu.be/UsnRQJxanVM?si=RI4EqEjIo_0Q_7v7",
    "https://youtu.be/s-Dq5FJEH10?si=fIhFAgx1JxL7UZu5",
    "https://youtu.be/ZfR_FP_7w00?si=N9WjmPEcREP53YOJ",
]

def main():
    with open(CSV_PATH, "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Name","Length","Genre","Language","Thematic"])

        for original_url in URLS:
            print(f"\nProcessing original link: {original_url}")

            # Step 1: Extract title from original
            with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
                info = ydl.extract_info(original_url, download=False)
                original_title = info.get('title')
            print(f"Original title: {original_title}")

            # Step 2: Search for better version
            better_url = search_youtube_for_better_version(original_title)
            if better_url:
                print(f"Found better version: {better_url}")
            else:
                print("No better version found, using original.")
                better_url = original_url

            # Step 3: Download better version
            title, duration, subtitle = download_song_and_metadata(better_url, OUTPUT_FOLDER)
            print(f"Downloaded: {title}")

            if subtitle and len(subtitle) > 100:
                genre, language, thematic = gpt_infer(subtitle[:2000])
                print(f"AI Inference: Genre={genre}, Language={language}, Thematic={thematic}")
            else:
                genre, language, thematic = "Unknown", "Unknown", "Unknown"
                print("No suitable subtitles/lyrics for inference.")

            writer.writerow([title, format_seconds_to_mmss(duration), genre, language, thematic])
            time.sleep(2)

    print(f"\nDone! Output in: {CSV_PATH}")

if __name__ == "__main__":
    main()



Processing original link: https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9
Original title: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas
Fallback Selected: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas
Found better version: https://www.youtube.com/watch?v=0zPjfX8PiGw
songs/Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas.webm
Downloaded: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas
No suitable subtitles/lyrics for inference.

Processing original link: https://youtu.be/ekr2nIex040?si=rRjc8j1PLoBzttmc
Original title: ROSÉ & Bruno Mars - APT. (Official Music Video)
Selected: ROSÉ & Bruno Mars - APT. (Official Music Video) from ROSÉ
Found better version: https://www.youtube.com/watch?v=ekr2nIex040
songs/ROSÉ & Bruno Mars - APT. (Official Music Video).webm
Downloaded: ROSÉ & Bruno Mars - APT. (Official Music Video)
AI Inference: Genre=Unknown, Language=Unknown, Thematic=Unknown

Processing original link: https://youtu.be/qrO

Selected: Toño Rosario -  Dale Vieja Dale (Official Video) from Toño Rosario
Found better version: https://www.youtube.com/watch?v=hal7rXfJj5o
songs/Toño Rosario -  Dale Vieja Dale (Official Video).webm
Downloaded: Toño Rosario -  Dale Vieja Dale (Official Video)
AI Inference: Genre=Unknown, Language=Unknown, Thematic=Unknown

Processing original link: https://youtu.be/L4IR6SkE_Aw?si=xQaKoIx4sxUTENBf


Original title: Digimon Frontier Opening - Español España (1080p HD) [2002]


Selected: Digimon Frontier Opening - Español España (1080p HD) [2002] from Openings Español España [HD]
Found better version: https://www.youtube.com/watch?v=L4IR6SkE_Aw
songs/Digimon Frontier Opening - Español España (1080p HD) [2002].webm
Downloaded: Digimon Frontier Opening - Español España (1080p HD) [2002]
AI Inference: Genre=Unknown, Language=Unknown, Thematic=Unknown

Processing original link: https://youtu.be/Lg_Pn45gyMs?si=HrnvZgkJD2FjyosH


Original title: Aventura - Ella y Yo (ft. Don Omar)


KeyboardInterrupt: 

In [7]:
import yt_dlp
import csv
import os
import openai
import time
import requests

def format_seconds_to_mmss(seconds):
    minutes, secs = divmod(int(seconds), 60)
    return f"{minutes:02d}:{secs:02d}"

api_key = "your_openai_api_key_here"

OUTPUT_FOLDER = "songs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CSV_PATH = "songs_info.csv"

client = openai.OpenAI(api_key=api_key)

def search_high_quality_version(title, artist):
    query = f"{title} {artist} official audio high quality"
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[
                {"role": "user", "content": f"Find the best quality official audio version of the song '{title}' by '{artist}'."}
            ],
            functions=[
                {
                    "name": "search",
                    "description": "Search the internet for the best quality official audio version of a song.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "query": {
                                "type": "string",
                                "description": "The search query."
                            }
                        },
                        "required": ["query"]
                    }
                }
            ],
            function_call={"name": "search", "arguments": {"query": query}}
        )
        # Extract the URL from the response
        search_results = response.choices[0].message.content.strip()
        return search_results  # This should be the URL of the best version
    except Exception as e:
        print(f"OpenAI API error during search: {e}")
        return None

def download_song_and_metadata(url, output_folder):
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en', 'es', 'auto'],
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'noplaylist': True,
        'skip_download': False,
        'forcefilename': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

        title = info.get('title')
        artist = info.get('uploader')
        duration = info.get('duration')
        subtitle_text = None

        if info.get('automatic_captions') or info.get('subtitles'):
            subs = info.get('subtitles') or info.get('automatic_captions')
            lang_code = next(iter(subs)) if subs else None
            if lang_code:
                sub_info = subs[lang_code][0]
                sub_url = sub_info['url']
                r = requests.get(sub_url)
                sub_filename = os.path.join(output_folder, f"{title}.vtt")
                with open(sub_filename, "wb") as f:
                    f.write(r.content)
                with open(sub_filename, "r", encoding="utf-8") as f:
                    vtt = f.read()
                lines = []
                for l in vtt.splitlines():
                    if "-->" in l or l.strip() == "" or l[0].isdigit():
                        continue
                    lines.append(l)
                subtitle_text = " ".join(lines)
        return title, artist, duration, subtitle_text

def gpt_infer(subtitle_text):
    prompt = (
        "Dada la letra de la canción que te estoy pasando, identifica lo siguiente:"
        "1. Tematica:  (Latino/Anime/JPOP/KPOP/Videojuegos/Clasicos/Conocidas/Otras), "
        "2. Idioma hablado"
        "3. Genero musical (e.g., pop, reggaeton, españolas, rap, classical, etc.). "
        "Return your answer as a CSV row in the format: Genero,Idioma,Tematica.\n\n"
        "Transcript:\n"
        f"{subtitle_text}\n"
        "CSV:"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "user", "content": prompt}],
        )
        txt = response.choices[0].message.content.strip()
        fields = txt.split(",")
        if len(fields) == 3:
            genre, language, thematic = [f.strip() for f in fields]
            return genre, language, thematic
        else:
            return "Unknown", "Unknown", "Unknown"
    except Exception as e:
        print(f"OpenAI API error: {e}")
        return "Unknown", "Unknown", "Unknown"

# List of original URLs
URLS = [
    "https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9",
    # Add more links here
]

def main():
    with open(CSV_PATH, "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Name", "Length", "Genre", "Language", "Thematic"])

        for original_url in URLS:
            print(f"\nProcessing original link: {original_url}")

            # Step 1: Extract metadata from original
            with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
                info = ydl.extract_info(original_url, download=False)
                original_title = info.get('title')
                original_artist = info.get('uploader')
            print(f"Original title: {original_title}, Artist: {original_artist}")

            # Step 2: Use OpenAI to search for high-quality version
            better_url = search_high_quality_version(original_title, original_artist)
            if better_url:
                print(f"Found better version: {better_url}")
            else:
                print("No better version found, using original.")
                better_url = original_url

            # Step 3: Download better version
            title, artist, duration, subtitle = download_song_and_metadata(better_url, OUTPUT_FOLDER)
            print(f"Downloaded: {title}")

            if subtitle and len(subtitle) > 100:
                genre, language, thematic = gpt_infer(subtitle[:2000])
                print(f"AI Inference: Genre={genre}, Language={language}, Thematic={thematic}")
            else:
                genre, language, thematic = "Unknown", "Unknown", "Unknown"
                print("No suitable subtitles/lyrics for inference.")

            writer.writerow([title, format_seconds_to_mmss(duration), genre, language, thematic])
            time.sleep(2)

    print(f"\nDone! Output in: {CSV_PATH}")

if __name__ == "__main__":
    main()



Processing original link: https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9


Original title: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas, Artist: Don Omar
OpenAI API error during search: Error code: 401 - {'error': {'message': 'Incorrect API key provided: your_ope************here. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}
No better version found, using original.
songs/Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas.webm


KeyboardInterrupt: 

In [ ]:
import yt_dlp
import csv
import os
import openai
import time
import requests

# ========== Basic Config ==========
api_key = "your_openai_api_key_here"

OUTPUT_FOLDER = "songs"
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

CSV_PATH = "songs_info.csv"

client = openai.OpenAI(api_key=api_key)

# ========== Utility Functions ==========
def format_seconds_to_mmss(seconds):
    minutes, secs = divmod(int(seconds), 60)
    return f"{minutes:02d}:{secs:02d}"

# ========== OpenAI Helpers ==========

def generate_youtube_search(metadata):
    """Use OpenAI to generate a better YouTube search query based on title/artist."""
    prompt = (
        f"Given this song metadata: Title: {metadata['title']}, Artist: {metadata['artist']}.\n"
        "Generate a YouTube search phrase to find the highest quality official or remastered version of this song. "
        "It should prefer official uploads, audio-only if possible, high quality, full song."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "user", "content": prompt}],
        )
        search_query = response.choices[0].message.content.strip()
        print(f"🔍 AI Search Query: {search_query}")
        return search_query
    except Exception as e:
        print(f"Error during OpenAI search query generation: {e}")
        return f"{metadata['title']} {metadata['artist']} official audio high quality"

def classify_song(metadata, lyrics):
    """Use OpenAI to classify the song based on metadata + lyrics."""
    prompt = (
        "Given the following metadata and lyrics of a song, classify:\n"
        "1. Genre (pop, reggaeton, classical, JPOP, etc)\n"
        "2. Language (English, Spanish, Japanese, etc)\n"
        "3. Thematic (Latino/Anime/JPOP/KPOP/Videojuegos/Clasicos/Conocidas/Otras)\n"
        "Metadata:\n"
        f"Title: {metadata['title']}\n"
        f"Artist: {metadata['artist']}\n\n"
        f"Lyrics:\n{lyrics}\n\n"
        "Return in CSV format: Genre, Language, Thematic"
    )
    try:
        response = client.chat.completions.create(
            model="gpt-4-1106-preview",
            messages=[{"role": "user", "content": prompt}],
        )
        txt = response.choices[0].message.content.strip()
        fields = txt.split(",")
        if len(fields) == 3:
            genre, language, thematic = [f.strip() for f in fields]
            return genre, language, thematic
        else:
            return "Unknown", "Unknown", "Unknown"
    except Exception as e:
        print(f"OpenAI API error: {e}")
        return "Unknown", "Unknown", "Unknown"

# ========== YouTube Helpers ==========

def search_best_youtube_link(search_query):
    """Search YouTube with yt_dlp and pick the top result."""
    search_url = f"ytsearch5:{search_query}"
    ydl_opts = {'quiet': True, 'skip_download': True}

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        results = ydl.extract_info(search_url, download=False)
        videos = results['entries']
        if not videos:
            return None
        return f"https://www.youtube.com/watch?v={videos[0]['id']}"

def download_song_and_lyrics(url, output_folder):
    """Download best audio + extract lyrics (subtitles)."""
    ydl_opts = {
        'format': 'bestaudio/best',
        'outtmpl': os.path.join(output_folder, '%(title)s.%(ext)s'),
        'writesubtitles': True,
        'writeautomaticsub': True,
        'subtitleslangs': ['en', 'es', 'auto'],
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'mp3',
            'preferredquality': '192',
        }],
        'quiet': True,
        'noplaylist': True,
        'skip_download': False,
        'forcefilename': True,
        'no_warnings': True,
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info = ydl.extract_info(url, download=True)

        title = info.get('title')
        artist = info.get('uploader')
        duration = info.get('duration')

        subtitle_text = None
        if info.get('automatic_captions') or info.get('subtitles'):
            subs = info.get('subtitles') or info.get('automatic_captions')
            lang_code = next(iter(subs)) if subs else None
            if lang_code:
                sub_info = subs[lang_code][0]
                sub_url = sub_info['url']
                r = requests.get(sub_url)
                sub_filename = os.path.join(output_folder, f"{title}.vtt")
                with open(sub_filename, "wb") as f:
                    f.write(r.content)
                with open(sub_filename, "r", encoding="utf-8") as f:
                    vtt = f.read()
                lines = []
                for l in vtt.splitlines():
                    if "-->" in l or l.strip() == "" or l[0].isdigit():
                        continue
                    lines.append(l)
                subtitle_text = " ".join(lines)
        return {'title': title, 'artist': artist, 'duration': duration}, subtitle_text

# ========== Main Program ==========


URLS = [
    "https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9",]

def main():
    with open(CSV_PATH, "w", newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["Name", "Length", "Genre", "Language", "Thematic"])

        for original_url in URLS:
            print(f"\n🎵 Processing: {original_url}")

            with yt_dlp.YoutubeDL({'quiet': True}) as ydl:
                info = ydl.extract_info(original_url, download=False)
                metadata = {
                    'title': info.get('title'),
                    'artist': info.get('uploader')
                }
            print(f"📋 Metadata extracted: {metadata}")

            # Step 1: Use OpenAI to generate a better search query
            search_query = generate_youtube_search(metadata)

            # Step 2: Search best version
            better_url = search_best_youtube_link(search_query)
            if not better_url:
                print("⚠️ No better version found. Using original URL.")
                better_url = original_url
            else:
                print(f"✅ Found better link: {better_url}")

            # Step 3: Download song and lyrics
            song_info, lyrics = download_song_and_lyrics(better_url, OUTPUT_FOLDER)

            # Step 4: Classify using OpenAI
            if lyrics and len(lyrics) > 50:
                genre, language, thematic = classify_song(song_info, lyrics[:3000])
                print(f"🧠 AI Classification: Genre={genre}, Language={language}, Thematic={thematic}")
            else:
                genre, language, thematic = "Unknown", "Unknown", "Unknown"
                print("⚠️ No suitable lyrics found for classification.")

            writer.writerow([song_info['title'], format_seconds_to_mmss(song_info['duration']), genre, language, thematic])
            time.sleep(2)

    print(f"\n✅ All done! Results saved to {CSV_PATH}")

if __name__ == "__main__":
    main()



Processing original link: https://youtu.be/0zPjfX8PiGw?si=jacWQmMQMga0Vmi9


Original title: Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas, Artist: Don Omar
OpenAI API error during search: 'NoneType' object has no attribute 'strip'
No better version found, using original.
songs\Don Omar, Juan Magán - No Sigue Modas Aka Ella No Sigue Modas.webm


ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location


DownloadError: ERROR: Postprocessing: ffprobe and ffmpeg not found. Please install or provide the path using --ffmpeg-location